In [5]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.linalg.DenseVector



import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.linalg.DenseVector


In [6]:
println(s"Current spark version is ${spark.version}")

Current spark version is 3.0.0


In [7]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = /home/jovyan/data/events-stream
modelPath: String = /home/jovyan/models/spark-ml-model
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [8]:
val a2 = udf { vr: org.apache.spark.ml.linalg.DenseVector => vr.toArray(1)}
val sameModel = PipelineModel.load(modelPath)
val predictionsDF = sameModel.transform(inputDF).withColumn("negative_response",a2($"probability"))
predictionsDF.printSchema()

root
 |-- tweet: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)
 |-- negative_response: double (nullable = false)



a2: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$2320/0x0000000840ec7840@178a7a2b,DoubleType,List(Some(class[value[0]: vector])),None,false,true)
sameModel: org.apache.spark.ml.PipelineModel = pipeline_f0c1e0306121
predictionsDF: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 5 more fields]


In [9]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

res4: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3a982cfd


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  negative_response|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|I've been stuck a...|[i've, been, stuc...|(1000,[17,48,109,...|[6.41449336869113...|[0.53454111405759...|       0.0|0.46545888594240553|
|@DawnofOURnight y...|[@dawnofournight,...|(1000,[17,202,330...|[5.68735915907620...|[0.47394659658968...|       1.0|  0.526053403410316|
|@IlanBr thought I...|[@ilanbr, thought...|(1000,[25,55,152,...|[6.14454544673131...|[0.51204545389427...|       0.0|0.48795454610572375|
|i just wanna go h...|[i, just, wanna, ...|(1000,[150,307,37...|[6.40195008969538...|[0.53349584080794...|       0.0|0.46650415919205135|
|@cristiano87 i th...|[@cristiano8


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  negative_response|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|feeling a little ...|[feeling, a, litt...|(1000,[17,115,248...|[6.34069868594258...|[0.52839155716188...|       0.0| 0.4716084428381177|
|I wish I could be...|[i, wish, i, coul...|(1000,[80,400,426...|[6.16436912999160...|[0.51369742749930...|       0.0| 0.4863025725006999|
|what a owwie ass ...|[what, a, owwie, ...|(1000,[118,467,49...|[6.172998374312,5...|[0.51441653119266...|       0.0|0.48558346880733333|
|@DHughesy unfortu...|[@dhughesy, unfor...|(1000,[162,166,22...|[5.75721745125212...|[0.47976812093767...|       1.0| 0.5202318790623227|
|UGH.  Pleasepleas...|[ugh., , pl


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  negative_response|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|  not exactly sure. |[not, exactly, su...|(1000,[96,173,405...|[5.96175435095738...|[0.49681286257978...|       1.0| 0.5031871374202178|
|http://twitpic.co...|[http://twitpic.c...|(1000,[59,71,152,...|[6.54031815054648...|[0.54502651254554...|       0.0| 0.4549734874544595|
|misses her already! |[misses, her, alr...|(1000,[20,112,912...|[5.85277960026398...|[0.48773163335533...|       1.0| 0.5122683666446681|
|Nevermind about g...|[nevermind, about...|(1000,[273,488,54...|[5.75721745125212...|[0.47976812093767...|       1.0| 0.5202318790623227|
|@BIGBULLCOW LMAO ...|[@bigbullco

In [ ]:
//MLUtils.convertVectorColumnsFromML(df)